In [1]:
from catboost import CatBoostRegressor


In [2]:

# Initialize data
train_data = [[1, 4, 5, 6], [4, 5, 6, 7], [30, 40, 50, 60]]

eval_data = [[2, 4, 6, 8], [1, 4, 50, 60]]

train_labels = [10, 20, 30]
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=2, learning_rate=1, depth=2)
# Fit model
model.fit(train_data, train_labels)
# Get predictions
preds = model.predict(eval_data)
print(preds)


0:	learn: 6.1237244	total: 47.1ms	remaining: 47.1ms
1:	learn: 4.5927933	total: 47.6ms	remaining: 0us
[15.625 18.125]


## Save Model
Model must have the file name `model.pkl` in current directory. This is hardcoded in `deploy.sh`

In [3]:
# save model
import pickle
with open('model.pkl', 'wb') as out:
    pickle.dump(model, out, protocol=3) # python3 default
  

## Deploy Endpoint

Take note that the following needs to be updated according:
- image name
- classic model location
- sagemaker execution role

In [12]:
! cp model.pkl local_test/test_dir/model/

In [25]:
!./deploy.sh sagemaker-inference-containers/test

a model.pkl
move: ./model.tar.gz to s3://wy-sandbox-singapore/sagemaker/model/sagemaker-inference-containers/test/input/model.tar.gz
342474125894
Login Succeeded
Login Succeeded
[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/4)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 84B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for 520713654638.dkr.ecr.ap-southeast-1.amaz  0.0s
 => [auth] sharing credentials for 520713654638.dkr.ecr.ap-southeast-1.am  0.0s
[+] Building 0.3s (3/4)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring

In [22]:
!aws sagemaker list-endpoints --name-contains rtc360-20210104-171844

{
    "Endpoints": [
        {
            "EndpointName": "rtc360-20210104-171844",
            "EndpointArn": "arn:aws:sagemaker:ap-southeast-1:342474125894:endpoint/rtc360-20210104-171844",
            "CreationTime": "2021-01-04T17:18:47.852000+08:00",
            "LastModifiedTime": "2021-01-04T17:25:02.847000+08:00",
            "EndpointStatus": "InService"
        }
    ]
}


In [16]:
!date

Mon Jan  4 17:16:09 +08 2021


## Inference


In [26]:
!echo "1,2,3,4\n5,6,7,8" >> test.csv

In [18]:
with open('test.csv', 'r') as f:
    payload = f.read()
    
print(payload)

1,2,3,4
5,6,7,8
1,2,3,4
5,6,7,8
1,2,3,4
5,6,7,8



In [23]:
# leaseplan-deploy-py3-20191015-055338 (iso match sklearn)
# leaseplan-deploy-py3-20191016-083601 (v2)

import io, boto3

sess = boto3.Session()
response = sess.client('sagemaker-runtime').invoke_endpoint(
    EndpointName='rtc360-20210104-171844', 
    Body=payload,
    ContentType='text/csv', 
)

results = response['Body'].read().decode('ascii')
print(results)


15.625
20.0
15.625
20.0
15.625
20.0

